# TFP_ASIF_china data preprocessing

This notebook has been generated on 2019-11-19 16:41 

The objective of this notebook is to create the dataframe to compute the TFP from ASIF dataset. The library to compute the TFP with R is [prodest](https://cran.r-project.org/web/packages/prodest/prodest.pdf). With Mac, there are some issues installing the package, so we move to the AWS R server.

## Global steps 

The global steps to construct the dataset are the following:

- Import the ASIF data from 98 to 07 and deflated data
- Construct deflated variables

## Data source 

The data source to construct the dataset are the following:


 ### Big Query Dataset 
 
 - asif_firm_china9807 
 ### Google Cloud Storage Dataset 
 
 ### Google Spreadsheet Dataset 
 
 Original source: https://feb.kuleuven.be/public/N07057/China/
 
 Original paper: https://docs.google.com/file/d/16agSbxO7cYuEn1v2bvw16ZRAx9gg7-Zm/edit
 
 - [benchmark_input_deflator_updated](https://docs.google.com/spreadsheets/d/1-9v7OuAFXFqkXpPJl-4xqfi1T0j4j0oG0vCkLMx6OK8) 
 - [benchmark_output_deflator](https://docs.google.com/spreadsheets/d/1O4tgEvhgytMUyNnXqwa0Cven3YfX9d_7TiO01b0-Zyg) 
 - [Brandt_Rawski investment deflator](https://docs.google.com/spreadsheets/d/1U8H799pbu2z6WF5aQgSzelqmiIrUpARIYtfrf5N4RfY)

## Destination

The new dataset is available from XXX

- GS: None
- GCS: TFP_ASIF_china.gz
- BG: TFP_ASIF_china

# Load Dataset


In [ ]:
from Fast_connectCloud import connector
from GoogleDrivePy.google_drive import connect_drive
from GoogleDrivePy.google_platform import connect_cloud_platform
import pandas as pd 
import numpy as np
import pandas_profiling

In [ ]:
gs = connector.open_connection(online_connection = False, 
	path_credential = '/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/')

service_gd = gs.connect_remote(engine = 'GS')
service_gcp = gs.connect_remote(engine = 'GCP')

gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])

project = 'valid-pagoda-132423'
gcp = connect_cloud_platform.connect_console(project = project,
											 service_account = service_gcp['GoogleCloudP'])


## Load asif_firm_china9807 from Google Big Query

Feel free to add description about the dataset or any usefull information.



In [ ]:

  query = (
          "SELECT id, year,cic, output, input, fa_net, employment "
            "FROM China.asif_firm_china9807Clean "

        )

df_asif_firm_china9807 = gcp.upload_data_from_bigquery(query = query, location = 'US')
df_asif_firm_china9807.head()


In [ ]:
df_asif_firm_china9807 = df_asif_firm_china9807.assign(year=lambda x: x['year'].astype('str'),
                                                       cic=lambda x: x['cic'].astype(
                                                           'str')
                                                       )
df_asif_firm_china9807.head()

In [ ]:
df_asif_firm_china9807.shape


## Load benchmark_input_deflator_updated from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset



In [ ]:

### Please go here https://docs.google.com/spreadsheets/d/1-9v7OuAFXFqkXpPJl-4xqfi1T0j4j0oG0vCkLMx6OK8
### To change the range

sheetid = '1-9v7OuAFXFqkXpPJl-4xqfi1T0j4j0oG0vCkLMx6OK8'
sheetname = 'benchmark_input_deflator_updated'

df_benchmark_input_deflator_updated = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)
df_benchmark_input_deflator_updated.head()



## Load benchmark_output_deflator from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset



In [ ]:

### Please go here https://docs.google.com/spreadsheets/d/1O4tgEvhgytMUyNnXqwa0Cven3YfX9d_7TiO01b0-Zyg
### To change the range

sheetid = '1O4tgEvhgytMUyNnXqwa0Cven3YfX9d_7TiO01b0-Zyg'
sheetname = 'benchmark_output_deflator'

df_benchmark_output_deflator = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)

df_benchmark_output_deflator.head()



## Load Brandt_Rawski investment deflator from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset



In [ ]:

### Please go here https://docs.google.com/spreadsheets/d/1U8H799pbu2z6WF5aQgSzelqmiIrUpARIYtfrf5N4RfY
### To change the range

sheetid = '1U8H799pbu2z6WF5aQgSzelqmiIrUpARIYtfrf5N4RfY'
sheetname = 'Brandt_Rawski investment deflator'

df_Brandt_Rawski_investment_deflator = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)
df_Brandt_Rawski_investment_deflator.head()


# Workflow

In this section, we will construct the dataset, and document each step of the workflow.

Please use the following format for the documentation:

- `##` Step 1: XXX
- `###` (optional) Underlying process description
- `##` Step 2: YYY
- `###` (optional) Underlying process description

Note: **You need to rename the last dataframe `df_final`**

## Create deflated variables:

- output deflated = value added / output deflated
- input deflated = input /deflated input
- capital deflated = fixed asset/ deflated capital

In [ ]:
df_asif_firm_china9807.shape

In [ ]:
df_final = (df_asif_firm_china9807
            .merge(df_benchmark_input_deflator_updated,
                   on=['cic', 'year'])
            .merge(df_benchmark_output_deflator,
                   on=['cic', 'year'])
            .merge(df_Brandt_Rawski_investment_deflator,
                   on=['year'])
            .assign(
                output_deflator=lambda x: x['output_deflator'].astype('float'),
                deflator_input=lambda x: x['deflator_input'].astype('float'),
                BR_deflator=lambda x: x['BR_deflator'].astype('float'),
                output_deflated=lambda x: x['input']/x['output_deflator'],
                input_deflated=lambda x: x['input']/x['deflator_input'],
                capital_deflated=lambda x: x['fa_net']/x['BR_deflator'],
            )
            #.drop(columns=['output', 'input', 'va', 'fa_net'])
            .set_index(['id', 'year', 'cic'])
            .loc[lambda x: (x > 0).all(1)]
            .reset_index()
            )
df_final["id_"] = pd.factorize(df_final["id"])[0]
df_final.sort_values(by = 'id_').head()

In [ ]:
output_ = df_final['output'].quantile([.05, .98])
input_ = df_final['input'].quantile([.05, .98])
#va_ = df_final['va'].quantile([.05, .98])
fa_net_ = df_final['fa_net'].quantile([.05, .98])
employment_ = df_final['employment'].quantile([.05, .98])

In [ ]:
### Bottom
outliers = True
if outliers:
    for n in [output_, input_, fa_net_, employment_]:
        name  = n.name
        df_final = df_final.loc[ lambda x:
        x[name] > n.iloc[0]
                ]
### Top
    for n in [output_, input_, fa_net_
         ]:
        name  = n.name
        df_final = df_final.loc[ lambda x:
        x[name] < n.iloc[1]
                ]
df_final.describe().style.format('{0:,.0f}')

In [ ]:
total_id_year = (df_final
 .groupby(['id'])['id']
 .count()
 .reset_index(name = 'count')
 .groupby('count')
 .count()
 .assign(total_obs = lambda x:
        x.index.get_level_values(0) * x['id'])
)
total_id_year['id'].plot.bar(title = 'count by # of year')

In [ ]:
#df_final = df_final.loc[lambda x: x['year'] > '2001']

In [ ]:
df_final.shape

In [ ]:
#df_final['year'].unique()

In [ ]:
sample = df_final['id'].sample(n=int(df_final['id'].nunique() * .4),
                               random_state=1).tolist()
len(sample)

In [ ]:
df_final_ = df_final.loc[lambda x :x['id'].isin(sample)]
df_final_.shape

In [ ]:
### First save locally: Sample
df_final_.to_csv(
	'TFP_ASIF_china_sample.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

In [ ]:
### First save locally: Full
df_final.to_csv(
	'TFP_ASIF_china.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

## Computation TFP

Computation is done with the `prodest` library.

```
# https://cran.r-project.org/web/packages/prodest/prodest.pdf

### Dependencies
# https://anaconda.org/conda-forge/parallel
# https://anaconda.org/r/r-matrix
# https://anaconda.org/conda-forge/r-rsolnp
# https://anaconda.org/russh/r-rcppde
# https://anaconda.org/conda-forge/r-aer
# https://anaconda.org/conda-forge/r-zip
# https://anaconda.org/bioconda/r-openxlsx
# https://anaconda.org/conda-forge/r-nloptr
# https://anaconda.org/conda-forge/r-pbkrtest
# https://anaconda.org/conda-forge/r-rio
# https://anaconda.org/conda-forge/r-lme4
```

The function `prodestLP` has the following arguments need to be passed. For simplicity, we keep only the deflated output, employment, capital and input. 

```
Y the vector of value added log output.
fX the vector/matrix/dataframe of log free variables.
sX the vector/matrix/dataframe of log state variables.
pX the vector/matrix/dataframe of log proxy variables.
cX the vector/matrix/dataframe of control variables. By default cX= NULL.
idvar the vector/matrix/dataframe identifying individual panels.
timevar the vector/matrix/dataframe identifying time
```

The TFP is the residual of the function

In [ ]:
library(prodest)
df <- read.csv("TFP_ASIF_china_sample.gz") 

In [ ]:
dim(df)

### Sample

To speed up time, we select half of the data. Coefficients won't change much

![](https://drive.google.com/uc?export=view&id=1W5rA8DYE8G4j5euJ0MwM_QpwjhQr2SK3)

In [ ]:
LP.fit <- prodestLP(Y = log(df$output),
                    fX = log(df$employment),
                    sX= log(df$capital),
                    pX = log(df$input),
                    idvar = df$id_,
                    timevar = df$year)

In [ ]:
summary(LP.fit)

![](https://drive.google.com/uc?export=view&id=1CtXSXp7rO_dgGbsJ-qvfmLir1B58WhG9)

In [ ]:
LP.fit1_def <- prodestLP(Y=log(df$output_deflated),
                          fX=log(df$employment),
                          sX=log(df$capital_deflated),
                          pX=log(df$input_deflated),
                          idvar=df$id_,
                          timevar=df$year)

In [ ]:
summary(LP.fit1_def)

### Full

to long to compute, avoid

In [ ]:
#df_ <- read.csv("TFP_ASIF_china.gz") 

In [ ]:
#LP.fit_ <- prodestLP(Y = log(df_$output),
#                    fX = log(df_$employment),
#                    sX= log(df_$capital),
#                    pX = log(df_$input),
#                    idvar = df_$id_,
#                    timevar = df_$year)
#summary(LP.fit_)

In [ ]:
#LP.fit1_def_ <- prodestLP(Y=log(df_$output_deflated),
#                          fX=log(df_$employment),
#                          sX=log(df_$capital_deflated),
#                          pX=log(df_$input_deflated),
#                          idvar=df_$id_,
#                          timevar=df_$year)
#summary(LP.fit1_def_)

In [ ]:
#head(df)

### add TFP

In [ ]:
df_ <- read.csv("TFP_ASIF_china.gz") 

In [ ]:
dim(df_)

In [ ]:
#### predict TFP
df_$tfp <- log(df_$output_deflated) - (log(df_$employment) * LP.fit@Estimates$pars[1] +
       log(df_$capital_deflated) * LP.fit@Estimates$pars[2])

In [ ]:
#### predict TFP
df_$tfp_def <- log(df_$output_deflated) - (log(df_$employment) * LP.fit1_def@Estimates$pars[1] +
       log(df_$capital_deflated) * LP.fit1_def@Estimates$pars[2])

In [ ]:
write.csv(df_, "TFP_computed_ASIF_china.csv", row.names=FALSE)

In [ ]:
### check summary stat
summary(df_)

In [ ]:
library(ggplot2)
# Basic density
ggplot(df_, aes(x=tfp)) + 
  geom_density() +
    labs(title="TFP density with LP method",
       x="TFP LP Method",
       y = "Density")

In [ ]:
ggplot(df_, aes(x=tfp_def)) + 
  geom_density() +
    labs(title="TFP density with LP method",
       x="TFP LP Method",
       y = "Density")

## Add TFP To ASIF 9807

In [ ]:
df_tfp = (pd.read_csv('TFP_computed_ASIF_china.csv', usecols = ['id', 'year', 'cic',
                                                              'tfp', 'tfp_def'])
          .assign(year = lambda x : x['year'].astype('str'),
                  cic = lambda x : x['cic'].astype('str')
                 )
          .merge(df_final)
         )
df_tfp.head()

In [ ]:
df_tfp.shape

# Profiling

In order to get a quick summary statistic of the data, we generate an HTML file with the profiling of the dataset we've just created. 

The profiling will be available at this URL after you commit a push to GitHub. 

**You need to rename the final dataframe `df_final` in the previous section to generate the profiling.**

In [ ]:
df_tfp['year'].unique()

In [ ]:
#### make sure the final dataframe is stored as df_final
### Overide the default value: 
#https://github.com/pandas-profiling/pandas-profiling/blob/master/pandas_profiling/config_default.yaml

profile = pandas_profiling.ProfileReport(df_tfp,
                                        check_correlation_pearson = False)
name_html = "TFP_ASIF_china.html"
profile.to_file(output_file=name_html)

# Upload to cloud

The dataset is ready to be shared with your colleagues. 





### Move to GCS and BigQuery

We move the dataset to the following:

- **bucket**: *NEED TO DEFINE*

- **Destination_blob**: *XXXXX/Processed_*
- **name**:  *TFP_ASIF_china.gz*
- **Dataset**: *China*

- **table**: *TFP_ASIF_china*

### GCS

We first need to save *TFP_ASIF_china* with `.gz` extension locally then we can move it
to GCS


In [ ]:
bucket_name = 'chinese_data'
destination_blob_name = 'Panel_china/Asif_panel_china/Processed_/TFP_asif_china.gz'

gcp.delete_blob(bucket_name = bucket_name,
                destination_blob_name= destination_blob_name)
gcp.delete_table(dataset_name = 'China', name_table = 'TFP_asif_china')

In [ ]:

### First save locally
df_tfp.to_csv(
	'TFP_ASIF_china.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

### Then upload to GCS
bucket_name = 'chinese_data'
destination_blob_name = 'Panel_china/Asif_panel_china/Processed_'
source_file_name = 'TFP_asif_china.gz'
gcp.upload_blob(bucket_name, destination_blob_name, source_file_name)


In [ ]:


### Move to bigquery
bucket_gcs ='chinese_data/Panel_china/Asif_panel_china/Processed_/TFP_asif_china.gz'
gcp.move_to_bq_autodetect(dataset_name= 'China',
							 name_table= 'TFP_asif_china',
							 bucket_gcs=bucket_gcs)


In [ ]:
import os
os.remove("TFP_computed_ASIF_china.csv")
os.remove("TFP_ASIF_china.gz")